In [1]:
import numpy as np
import numba as nb
import Ivvsc
import importlib
import time

In [2]:
data35 = np.load("Run35.npz", allow_pickle=True)
settings = data35['settings'].item()
raw = data35['raw'].item()
prob = data35['prob'].item()

In [3]:
N = 201
e_max = 10
B = 2

p = np.linspace(0, e_max, N)
delta_p = p[1] - p[0]

In [28]:
def calc_c(data, dN, p, e_max, B, dp, output_filename = "data"):
    C_full = np.zeros_like(data)
    
    N_total = len(data)
    
    N_steps = N_total // dN
    
    time_array = np.zeros(N_steps)
    
    for i in range(N_steps):
        beg = time.time()
        for j in range(dN):
            index = i * dN + j
            print("Time step {} / {}".format(index+1, N_total))
            C_full[index, :, :] = Ivvsc.C_Ivvsc(data[index,:,:], p, e_max, B, dp)
        np.savez(output_filename + "-" + str(i), C = C_full[:(i+1)*dN,:,:], p = p)
        print("File {}-{}.npz number {} of {}.".format(output_filename, i, i+1, N_steps))
        time_array[i] = (time.time() - beg) * 1000000
        time_string = "{:.1f}\% done. total time: {:.2f} hrs ({:.1f} min for last file)".format(100*(index+1)/N_total, np.sum(time_array)/60/60, time_array[i]/60)
        print(time_string)
        
    if index != len(data) - 1:
        i = N_steps
        for k in range(index+1, len(data)):
            C_full[k, :, :] = Ivvsc.C_Ivvsc(data[k,:,:], p, e_max, B, dp)
            
        
    np.savez(output_filename, C = C_full[:(i+1)*dN,:,:], p = p)
            
    return

In [31]:
calc_c(raw['nu3D'], 50, p, e_max, B, delta_p, output_filename="test")

Time step 1 / 725
Time step 2 / 725
Time step 3 / 725
Time step 4 / 725
Time step 5 / 725
Time step 6 / 725
Time step 7 / 725
Time step 8 / 725
Time step 9 / 725
Time step 10 / 725
Time step 11 / 725
Time step 12 / 725
Time step 13 / 725
Time step 14 / 725
Time step 15 / 725
Time step 16 / 725
Time step 17 / 725
Time step 18 / 725
Time step 19 / 725
Time step 20 / 725
Time step 21 / 725
Time step 22 / 725
Time step 23 / 725
Time step 24 / 725
Time step 25 / 725
Time step 26 / 725
Time step 27 / 725
Time step 28 / 725
Time step 29 / 725
Time step 30 / 725
Time step 31 / 725
Time step 32 / 725
Time step 33 / 725
Time step 34 / 725
Time step 35 / 725
Time step 36 / 725
Time step 37 / 725
Time step 38 / 725
Time step 39 / 725
Time step 40 / 725
Time step 41 / 725
Time step 42 / 725
Time step 43 / 725
Time step 44 / 725
Time step 45 / 725
Time step 46 / 725
Time step 47 / 725
Time step 48 / 725
Time step 49 / 725
Time step 50 / 725
File test-0.npz number 1 of 14.
6.9\% done. total time: 0.3